# Chapter 10: Working with Data (ORM)

We would be using ORM models with Python dataclass integration. These are copied
from chapter 8.


In [164]:
import logging

DEBUG = True  # control logs with this variable

if not DEBUG:
    logging.disable(logging.INFO)

Postponed Evaluation of Annotations:

In [165]:
from __future__ import annotations  # PEP-563

Necessary imports:

In [166]:
import datetime
import enum
from decimal import Decimal
from typing import Annotated

from sqlalchemy import (CheckConstraint, ForeignKey, Index, Numeric, String,
                        create_engine)
from sqlalchemy.ext.associationproxy import AssociationProxy, association_proxy
from sqlalchemy.orm import (DeclarativeBase, Mapped, MappedAsDataclass,
                            mapped_column, query_expression, relationship,
                            sessionmaker)

SQLite database:

In [167]:
DATABASE_URL = "sqlite+pysqlite:///store.db"

In [168]:
engine = create_engine(
    DATABASE_URL,
    echo=True,
)

Session maker factory:

In [169]:
SessionMaker = sessionmaker(
    bind=engine,
    expire_on_commit=True,  # default
    autoflush=True,  # default
)

Base class with Python dataclass integration:

In [170]:
class Base(MappedAsDataclass, DeclarativeBase):
    pass

Product type enumeration:

In [171]:
class ProductType(enum.Enum):
    PHONE = 0
    ACCESSORY = 1
    OTHER = 2

Re-usable types:

In [172]:
int_pk = Annotated[
    int,
    mapped_column(
        primary_key=True,
    )
]
date_auto = Annotated[
    datetime.date,
    mapped_column(
        default=datetime.date.today,
    )
]
timestamp_auto = Annotated[
    datetime.datetime,
    mapped_column(
        default=datetime.datetime.now,
    ),
]
str_127 = Annotated[
    str,
    mapped_column(
        String(127),
    )
]
str_255 = Annotated[
    str,
    mapped_column(
        String(255),
    )
]
num_12_2 = Annotated[
    Decimal,
    mapped_column(
        Numeric(12, 2),
    )
]

Employee:

In [173]:
class Employee(Base):
    __tablename__ = "employee"

    employee_id: Mapped[int_pk] = mapped_column(init=False)

    manager_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    name: Mapped[str_127] = mapped_column(
        CheckConstraint(
            "length(name)>0",
            name="name_length_must_be_at_least_one_character",
        ),
        default="",
    )
    is_manager: Mapped[bool] = mapped_column(default=False)
    hire_date: Mapped[date_auto] = mapped_column(default=None)

    # self-referential relationship: manager/employees
    manager: Mapped[Employee] = relationship(
        back_populates="employees",
        remote_side=[employee_id],
        init=False,
        repr=False,
    )
    employees: Mapped[list[Employee]] = relationship(
        back_populates="manager",
        init=False,
        repr=False,
    )

    orders: Mapped[list[Order]] = relationship(
        back_populates="employee",
        init=False,
        repr=False,
    )

Customer:

In [174]:
class Customer(Base):
    __tablename__ = "customer"

    customer_id: Mapped[int_pk] = mapped_column(init=False)

    first_name: Mapped[str_127]
    last_name: Mapped[str_127]
    address: Mapped[str_255] = mapped_column(
        # deferring will cause this column to be
        # updated by session.merge() every time!
        deferred=True,  # lazy load this column
        deferred_group="customer_attributes",
    )
    email: Mapped[str_127] = mapped_column(unique=True)

    order_count: Mapped[int] = query_expression(repr=False)

    __table_args__ = (
        Index("customer_full_name", "first_name", "last_name"),
    )

    orders: Mapped[list[Order]] = relationship(
        # lazy loading (default): "select" (named since a SELECT is emitted)
        # eager loading: "selectin", "joined"
        lazy="select",
        back_populates="customer",
        init=False,
        repr=False,
        order_by="desc(Order.order_id)",
    )

Order:

In [175]:
class Order(Base):
    __tablename__ = "order"

    order_id: Mapped[int_pk] = mapped_column(init=False)

    customer_id: Mapped[int] = mapped_column(
        ForeignKey("customer.customer_id"),
        default=None,
    )
    employee_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    order_datetime: Mapped[timestamp_auto] = mapped_column(init=False)
    is_shipped: Mapped[bool] = mapped_column(default=False)

    customer: Mapped[Customer] = relationship(
        back_populates="orders",
        init=False,
        repr=False,
    )
    employee: Mapped[Employee | None] = relationship(
        back_populates="orders",
        init=False,
        repr=False,
    )

    # relationship with associative table:
    order_details: Mapped[list[OrderDetail]] = relationship(
        back_populates="order",
        init=False,
        repr=False,
        # cascade delete and delete orphan (matches database FK constraint)
        cascade="all, delete-orphan",
        # indicates cascade rule already set on FK
        # and does not load unloaded children
        passive_deletes=True,
    )

    # many-to-many relationship with `Product`, bypassing `OrderDetail`
    products: Mapped[list[Product]] = relationship(
        init=False,
        repr=False,
        secondary="order_detail",
        back_populates="orders",
        viewonly=True,  # avoid conflicting changes between relations
    )

    product_names: AssociationProxy[list[str]] = association_proxy(
        "products",
        "product_name",
        init=False,
        repr=False,
    )

OrderDetail:

In [176]:
class OrderDetail(Base):
    """
    Association object pattern.
    This uses the associative table between Order and Product.
    """
    __tablename__ = "order_detail"

    order_id: Mapped[int] = mapped_column(
        # database side: ON DELETE CASCADE
        ForeignKey("order.order_id", ondelete="CASCADE"),
        primary_key=True,
        default=None,
    )
    product_id: Mapped[int] = mapped_column(
        ForeignKey("product.product_id"),
        primary_key=True,
        default=None,
    )

    quantity: Mapped[int] = mapped_column(
        CheckConstraint(
            "quantity>0",
            name="num_of_ordered_item_must_be_positive",
        ),
        default=1,
    )

    order: Mapped[Order] = relationship(
        back_populates="order_details",
        init=False,
        repr=False,
    )

    product: Mapped[Product] = relationship(
        back_populates="order_details",
        init=False,
        repr=False,
    )

Product:

In [177]:
class Product(Base, repr=False):  # type: ignore
    __tablename__ = "product"

    product_id: Mapped[int_pk] = mapped_column(init=False)

    product_name: Mapped[str_255] = mapped_column(index=True)
    unit_price: Mapped[num_12_2] = mapped_column(
        CheckConstraint("unit_price>0"))
    units_in_stock: Mapped[int] = mapped_column(
        CheckConstraint("units_in_stock>=0"),
        default=0,
    )
    # enum: directly use `ProductType` here:
    type: Mapped[ProductType] = mapped_column(
        default=ProductType.OTHER,
    )

    order_details: Mapped[list[OrderDetail]] = relationship(
        init=False,
        repr=False,
        back_populates="product",
    )

    # many-to-many relationship to `Order`, bypassing `OrderDetail`
    orders: Mapped[list[Order]] = relationship(
        init=False,
        secondary="order_detail",
        back_populates="products",
        viewonly=True,
    )

    # customize repr:
    def __repr__(self) -> str:
        return (
            "Product("
            f"product_id={self.product_id}, "
            f"product_name='{self.product_name}', "
            f"unit_price={self.unit_price}, "
            f"units_in_stock={self.units_in_stock}, "
            f"type='{self.type.name.lower()}'"
            ")"
        )

Drop and re-create all tables:

In [178]:
Base.metadata.drop_all(engine)

2024-03-28 17:45:17,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,107 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-28 17:45:17,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,109 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-28 17:45:17,109 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,110 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order")
2024-03-28 17:45:17,111 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,111 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order_detail")
2024-03-28 17:45:17,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,113 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-28 17:45:17,113 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,114 INFO sqlalchemy.engine.Engine 
DROP TABLE order_detail
2024-03-28 17:45:17,114 INFO sqlalchemy.engine.Engine [no key 0.00036s

In [179]:
Base.metadata.create_all(engine)

2024-03-28 17:45:17,137 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,138 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-28 17:45:17,139 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,139 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2024-03-28 17:45:17,140 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,140 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-28 17:45:17,141 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,141 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customer")
2024-03-28 17:45:17,142 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,142 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order")
2024-03-28 17:45:17,142 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-28 17:45:17,143 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("order")
2024-03-28 17:45:17,143 INFO sqlalchemy.engine.Engine [raw sql] ()

## Creating Data

In [180]:
session = SessionMaker()

Create employees:

Adding Alice and her two subordinates Bob and Cathy:

In [181]:
from datetime import date, timedelta

In [182]:
alice = Employee(
    name="Alice",
    is_manager=True,
    hire_date=date.today() - timedelta(days=1),
)
session.add(alice)

bob = Employee(name="Bob")
cathy = Employee(name="Cathy")

alice.employees.extend([bob, cathy])

session.flush()

2024-03-28 17:45:17,201 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,203 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,203 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (None, 'Alice', 1, '2024-03-27')
2024-03-28 17:45:17,205 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?) RETURNING employee_id
2024-03-28 17:45:17,205 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues) 1/2 (ordered; batch not supported)] (1, 'Bob', 0, '2024-03-28')
2024-03-28 17:45:17,206 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?) RETURNING employee_id
2024-03-28 17:45:17,206 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] (1, 'Cathy', 0, '2024-03-28')


Continue adding other employee data:

In [183]:
# Louis and his subordinate Lilly
louis = Employee(
    name="Louis",
    is_manager=True,
    hire_date=date.today() - timedelta(days=30),
)
session.add(louis)

lilly = Employee(
    name="Lilly",
    hire_date=date.today() - timedelta(days=20),
)
# use the manager relationship
lilly.manager = louis
session.add(lilly)

session.flush()

# employee Alice
alice2 = Employee(name="Alice")
session.add(alice2)

session.commit()

2024-03-28 17:45:17,216 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,216 INFO sqlalchemy.engine.Engine [cached since 0.01376s ago] (None, 'Louis', 1, '2024-02-27')
2024-03-28 17:45:17,217 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,217 INFO sqlalchemy.engine.Engine [cached since 0.01496s ago] (4, 'Lilly', 0, '2024-03-08')
2024-03-28 17:45:17,219 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,219 INFO sqlalchemy.engine.Engine [generated in 0.00042s] (None, 'Alice', 0, '2024-03-28')
2024-03-28 17:45:17,220 INFO sqlalchemy.engine.Engine COMMIT


In [184]:
from sqlalchemy import select

In [185]:
employees = session.scalars(select(Employee))

2024-03-28 17:45:17,290 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,292 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee
2024-03-28 17:45:17,292 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()


In [186]:
for employee in employees:
    print(employee)

Employee(employee_id=1, manager_id=None, name='Alice', is_manager=True, hire_date=datetime.date(2024, 3, 27))
Employee(employee_id=2, manager_id=1, name='Bob', is_manager=False, hire_date=datetime.date(2024, 3, 28))
Employee(employee_id=3, manager_id=1, name='Cathy', is_manager=False, hire_date=datetime.date(2024, 3, 28))
Employee(employee_id=4, manager_id=None, name='Louis', is_manager=True, hire_date=datetime.date(2024, 2, 27))
Employee(employee_id=5, manager_id=4, name='Lilly', is_manager=False, hire_date=datetime.date(2024, 3, 8))
Employee(employee_id=6, manager_id=None, name='Alice', is_manager=False, hire_date=datetime.date(2024, 3, 28))


Product data:

In [187]:
PRODUCT_DATA = [
    {
        "product_name": "phone",
        "unit_price": 300.0,
        "units_in_stock": 5,
        "type": ProductType.PHONE,
    },
    {
        "product_name": "phone screen protector",
        "unit_price": 9.50,
        "units_in_stock": 10,
        "type": ProductType.ACCESSORY,
    },
    {
        "product_name": "headphone",
        "unit_price": 25.99,
        "units_in_stock": 10,
        "type": ProductType.ACCESSORY,
    },
    {
        "product_name": "digital camera",
        "unit_price": 45.99,
        "units_in_stock": 5,
        "type": ProductType.OTHER,
    },
    {
        "product_name": "memory card 256GB",
        "unit_price": 21.99,
        "units_in_stock": 1,
        "type": ProductType.ACCESSORY,
    },
]

In [188]:
from sqlalchemy import insert

We need to save product PKs for later use:

In [189]:
product_ids = []

Create products with ORM bulk insert mode:

In [190]:
products = session.scalars(
    insert(Product).returning(
        Product,
        # to ensure the order (might impact performance)
        sort_by_parameter_order=True,
    ),
    PRODUCT_DATA
)

2024-03-28 17:45:17,319 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?) RETURNING product_id, product_name, unit_price, units_in_stock, type
2024-03-28 17:45:17,319 INFO sqlalchemy.engine.Engine [generated in 0.00015s (insertmanyvalues) 1/5 (ordered; batch not supported)] ('phone', 300.0, 5, 'PHONE')
2024-03-28 17:45:17,320 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?) RETURNING product_id, product_name, unit_price, units_in_stock, type
2024-03-28 17:45:17,321 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/5 (ordered; batch not supported)] ('phone screen protector', 9.5, 10, 'ACCESSORY')
2024-03-28 17:45:17,321 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?) RETURNING product_id, product_name, unit_price, units_in_stock, type
2024-03-28 17:45:17,322 INFO sqlalchemy.engine

In [191]:
for product in products:
    print(product)
    product_ids.append(product.product_id)

Product(product_id=1, product_name='phone', unit_price=300.00, units_in_stock=5, type='phone')
Product(product_id=2, product_name='phone screen protector', unit_price=9.50, units_in_stock=10, type='accessory')
Product(product_id=3, product_name='headphone', unit_price=25.99, units_in_stock=10, type='accessory')
Product(product_id=4, product_name='digital camera', unit_price=45.99, units_in_stock=5, type='other')
Product(product_id=5, product_name='memory card 256GB', unit_price=21.99, units_in_stock=1, type='accessory')


In [192]:
session.commit()

2024-03-28 17:45:17,333 INFO sqlalchemy.engine.Engine COMMIT


In [193]:
product_ids

[1, 2, 3, 4, 5]

Create customers:

In [194]:
CUSTOMER_DATA = [
    {
        "first_name": "Alex",
        "last_name": "Smith",
        "address": "618 Oak Lane, CA",
        "email": "alex_smith@test.com"
    },
    {
        "first_name": "Mary",
        "last_name": "Taylor",
        "address": "200-139 Jefferson Street, NY",
        "email": "mary_taylor@test.com"
    },
]

In [195]:
customer_ids = [] # we need this later

In [196]:
for data in CUSTOMER_DATA:
    customer = Customer(**data)
    session.add(customer)
    session.flush()  # flush to generate PKs immediately
    customer_ids.append(customer.customer_id)

2024-03-28 17:45:17,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,355 INFO sqlalchemy.engine.Engine INSERT INTO customer (first_name, last_name, address, email) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,356 INFO sqlalchemy.engine.Engine [generated in 0.00039s] ('Alex', 'Smith', '618 Oak Lane, CA', 'alex_smith@test.com')
2024-03-28 17:45:17,357 INFO sqlalchemy.engine.Engine INSERT INTO customer (first_name, last_name, address, email) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,357 INFO sqlalchemy.engine.Engine [cached since 0.001823s ago] ('Mary', 'Taylor', '200-139 Jefferson Street, NY', 'mary_taylor@test.com')


In [197]:
session.commit()

2024-03-28 17:45:17,361 INFO sqlalchemy.engine.Engine COMMIT


In [198]:
customer_ids

[1, 2]

In [199]:
customers = session.scalars(select(Customer))

2024-03-28 17:45:17,373 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,375 INFO sqlalchemy.engine.Engine SELECT customer.customer_id, customer.first_name, customer.last_name, customer.email 
FROM customer
2024-03-28 17:45:17,375 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ()


In [200]:
for customer in customers:
    print(customer)

2024-03-28 17:45:17,381 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,381 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (1,)
Customer(customer_id=1, first_name='Alex', last_name='Smith', address='618 Oak Lane, CA', email='alex_smith@test.com')
2024-03-28 17:45:17,382 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,382 INFO sqlalchemy.engine.Engine [cached since 0.001731s ago] (2,)
Customer(customer_id=2, first_name='Mary', last_name='Taylor', address='200-139 Jefferson Street, NY', email='mary_taylor@test.com')


Creating orders and order details:

In [201]:
# customer PKs:
alex_id, mary_id = customer_ids

# product PKs:
(
    phone_id,
    screen_protector_id,
    headphone_id,
    camera_id,
    memory_card_id,
) = product_ids

In [202]:
# Alex's first order:
order1 = Order(customer_id=alex_id)
order1.order_details.extend([
    OrderDetail(product_id=phone_id),
    OrderDetail(product_id=screen_protector_id),
    OrderDetail(product_id=headphone_id),
])
session.add(order1)
session.flush()


2024-03-28 17:45:17,393 INFO sqlalchemy.engine.Engine INSERT INTO "order" (customer_id, employee_id, order_datetime, is_shipped) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,394 INFO sqlalchemy.engine.Engine [generated in 0.00073s] (1, None, '2024-03-28 17:45:17.393781', 0)
2024-03-28 17:45:17,395 INFO sqlalchemy.engine.Engine INSERT INTO order_detail (order_id, product_id, quantity) VALUES (?, ?, ?)
2024-03-28 17:45:17,396 INFO sqlalchemy.engine.Engine [generated in 0.00033s] [(1, 1, 1), (1, 2, 1), (1, 3, 1)]


In [203]:
# Alex's second order:
order2 = Order(customer_id=alex_id)
OrderDetail(product_id=memory_card_id).order = order2
session.add(order2)
session.flush()

2024-03-28 17:45:17,401 INFO sqlalchemy.engine.Engine INSERT INTO "order" (customer_id, employee_id, order_datetime, is_shipped) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,402 INFO sqlalchemy.engine.Engine [cached since 0.008693s ago] (1, None, '2024-03-28 17:45:17.401588', 0)
2024-03-28 17:45:17,403 INFO sqlalchemy.engine.Engine INSERT INTO order_detail (order_id, product_id, quantity) VALUES (?, ?, ?)
2024-03-28 17:45:17,403 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (2, 5, 1)


In [204]:
# Mary's first order, or the third order:
order3 = Order(customer_id=mary_id)
session.add(order3)
session.flush()  # flush to generate ID
od31 = OrderDetail(product_id=camera_id, order_id=order3.order_id)
od32 = OrderDetail(
    product_id=memory_card_id,
    order_id=order3.order_id,
    quantity=2,
)
session.add_all([od31, od32])

2024-03-28 17:45:17,409 INFO sqlalchemy.engine.Engine INSERT INTO "order" (customer_id, employee_id, order_datetime, is_shipped) VALUES (?, ?, ?, ?)
2024-03-28 17:45:17,410 INFO sqlalchemy.engine.Engine [cached since 0.01682s ago] (2, None, '2024-03-28 17:45:17.409931', 0)


In [205]:
session.commit()

2024-03-28 17:45:17,415 INFO sqlalchemy.engine.Engine INSERT INTO order_detail (order_id, product_id, quantity) VALUES (?, ?, ?)
2024-03-28 17:45:17,416 INFO sqlalchemy.engine.Engine [cached since 0.02035s ago] [(3, 4, 1), (3, 5, 2)]
2024-03-28 17:45:17,416 INFO sqlalchemy.engine.Engine COMMIT


Checking orders:

In [206]:
orders = session.scalars(select(Order))

2024-03-28 17:45:17,423 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,424 INFO sqlalchemy.engine.Engine SELECT "order".order_id, "order".customer_id, "order".employee_id, "order".order_datetime, "order".is_shipped 
FROM "order"
2024-03-28 17:45:17,425 INFO sqlalchemy.engine.Engine [generated in 0.00055s] ()


In [207]:
for order in orders:
    print(order)

Order(order_id=1, customer_id=1, employee_id=None, order_datetime=datetime.datetime(2024, 3, 28, 17, 45, 17, 393781), is_shipped=False)
Order(order_id=2, customer_id=1, employee_id=None, order_datetime=datetime.datetime(2024, 3, 28, 17, 45, 17, 401588), is_shipped=False)
Order(order_id=3, customer_id=2, employee_id=None, order_datetime=datetime.datetime(2024, 3, 28, 17, 45, 17, 409931), is_shipped=False)


Checking order details:

In [208]:
ods = session.scalars(select(OrderDetail))

2024-03-28 17:45:17,434 INFO sqlalchemy.engine.Engine SELECT order_detail.order_id, order_detail.product_id, order_detail.quantity 
FROM order_detail
2024-03-28 17:45:17,434 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ()


In [209]:
for od in ods:
    print(od)

OrderDetail(order_id=1, product_id=1, quantity=1)
OrderDetail(order_id=1, product_id=2, quantity=1)
OrderDetail(order_id=1, product_id=3, quantity=1)
OrderDetail(order_id=2, product_id=5, quantity=1)
OrderDetail(order_id=3, product_id=4, quantity=1)
OrderDetail(order_id=3, product_id=5, quantity=2)


## Reading Data

Query for products:

In [210]:
stmt = select(Product)

In [211]:
print(stmt)

SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product


Note that tuples containing a single element are returned when `Session.execute()` is used:

In [212]:
result = session.execute(stmt)

2024-03-28 17:45:17,453 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product
2024-03-28 17:45:17,453 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()


In [213]:
result

In [214]:
for row in result:
    print(row)

(Product(product_id=1, product_name='phone', unit_price=300.00, units_in_stock=5, type='phone'),)
(Product(product_id=2, product_name='phone screen protector', unit_price=9.50, units_in_stock=10, type='accessory'),)
(Product(product_id=3, product_name='headphone', unit_price=25.99, units_in_stock=10, type='accessory'),)
(Product(product_id=4, product_name='digital camera', unit_price=45.99, units_in_stock=5, type='other'),)
(Product(product_id=5, product_name='memory card 256GB', unit_price=21.99, units_in_stock=1, type='accessory'),)


To directly obtain `Product` instances instead of tuples, you can use the `Session.scalars()` method:

In [215]:
for product in session.scalars(stmt):
    print(product)

2024-03-28 17:45:17,469 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product
2024-03-28 17:45:17,470 INFO sqlalchemy.engine.Engine [cached since 0.01726s ago] ()
Product(product_id=1, product_name='phone', unit_price=300.00, units_in_stock=5, type='phone')
Product(product_id=2, product_name='phone screen protector', unit_price=9.50, units_in_stock=10, type='accessory')
Product(product_id=3, product_name='headphone', unit_price=25.99, units_in_stock=10, type='accessory')
Product(product_id=4, product_name='digital camera', unit_price=45.99, units_in_stock=5, type='other')
Product(product_id=5, product_name='memory card 256GB', unit_price=21.99, units_in_stock=1, type='accessory')


Using `Result.all()`:

Using `Session.execute()`:

In [216]:
stmt = select(Product)
result = session.execute(stmt)

2024-03-28 17:45:17,476 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product
2024-03-28 17:45:17,476 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ()


A list of tuples is returned:

In [217]:
result.all()

[(Product(product_id=1, product_name='phone', unit_price=300.00, units_in_stock=5, type='phone'),),
 (Product(product_id=2, product_name='phone screen protector', unit_price=9.50, units_in_stock=10, type='accessory'),),
 (Product(product_id=3, product_name='headphone', unit_price=25.99, units_in_stock=10, type='accessory'),),
 (Product(product_id=4, product_name='digital camera', unit_price=45.99, units_in_stock=5, type='other'),),
 (Product(product_id=5, product_name='memory card 256GB', unit_price=21.99, units_in_stock=1, type='accessory'),)]

Using `Session.scalars()`:

In [218]:
products = session.scalars(stmt)

2024-03-28 17:45:17,486 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product
2024-03-28 17:45:17,487 INFO sqlalchemy.engine.Engine [cached since 0.01122s ago] ()


In [219]:
products

A list of products is returned:

In [220]:
products.all()

[Product(product_id=1, product_name='phone', unit_price=300.00, units_in_stock=5, type='phone'),
 Product(product_id=2, product_name='phone screen protector', unit_price=9.50, units_in_stock=10, type='accessory'),
 Product(product_id=3, product_name='headphone', unit_price=25.99, units_in_stock=10, type='accessory'),
 Product(product_id=4, product_name='digital camera', unit_price=45.99, units_in_stock=5, type='other'),
 Product(product_id=5, product_name='memory card 256GB', unit_price=21.99, units_in_stock=1, type='accessory')]

Using `Session.scalar()` to retrieve the first scalar:

In [221]:
product = session.scalar(stmt)

2024-03-28 17:45:17,500 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product
2024-03-28 17:45:17,501 INFO sqlalchemy.engine.Engine [cached since 0.0253s ago] ()


In [222]:
product

Product(product_id=1, product_name='phone', unit_price=300.00, units_in_stock=5, type='phone')

Selecting Specific Columns:

In [223]:
stmt = select(Product.product_name, Product.unit_price)

In [224]:
print(stmt)

SELECT product.product_name, product.unit_price 
FROM product


In [225]:
for row in session.execute(stmt):
    print(f"{row.product_name:<22}: ${row.unit_price}")

2024-03-28 17:45:17,518 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price 
FROM product
2024-03-28 17:45:17,519 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()
phone                 : $300.00
phone screen protector: $9.50
headphone             : $25.99
digital camera        : $45.99
memory card 256GB     : $21.99


Sorting:

In [226]:
stmt = stmt.order_by(Product.unit_price.desc())

In [227]:
print(stmt)

SELECT product.product_name, product.unit_price 
FROM product ORDER BY product.unit_price DESC


In [228]:
for row in session.execute(stmt):
    print(f"{row.product_name:<22}: ${row.unit_price}")

2024-03-28 17:45:17,532 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price 
FROM product ORDER BY product.unit_price DESC
2024-03-28 17:45:17,533 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()
phone                 : $300.00
digital camera        : $45.99
headphone             : $25.99
memory card 256GB     : $21.99
phone screen protector: $9.50


Offsetting and Limiting (pagination):

In [229]:
stmt = stmt.offset((2 - 1) * 2).limit(2)

In [230]:
print(stmt)

SELECT product.product_name, product.unit_price 
FROM product ORDER BY product.unit_price DESC
 LIMIT :param_1 OFFSET :param_2


In [231]:
print(stmt.compile().params)

{'param_1': 2, 'param_2': 2}


In [232]:
for row in session.execute(stmt):
    print(f"{row.product_name:<22}: ${row.unit_price}")

2024-03-28 17:45:17,551 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price 
FROM product ORDER BY product.unit_price DESC
 LIMIT ? OFFSET ?
2024-03-28 17:45:17,552 INFO sqlalchemy.engine.Engine [generated in 0.00089s] (2, 2)
headphone             : $25.99
memory card 256GB     : $21.99


Filtering (by product type):

Phones (filtering using `Select.where()`):

In [233]:
stmt = (
    select(Product.product_name)
    .where(Product.type == ProductType.PHONE)
)

In [234]:
print(stmt)

SELECT product.product_name 
FROM product 
WHERE product.type = :type_1


In [235]:
session.scalars(stmt).all()

2024-03-28 17:45:17,565 INFO sqlalchemy.engine.Engine SELECT product.product_name 
FROM product 
WHERE product.type = ?
2024-03-28 17:45:17,566 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ('PHONE',)


['phone']

Accessories (filtering using `Select.filter_by()`):

In [236]:
stmt = (
    select(Product.product_name)
    .filter_by(type=ProductType.ACCESSORY)
)

In [237]:
print(stmt)

SELECT product.product_name 
FROM product 
WHERE product.type = :type_1


In [238]:
session.scalars(stmt).all()

2024-03-28 17:45:17,579 INFO sqlalchemy.engine.Engine SELECT product.product_name 
FROM product 
WHERE product.type = ?
2024-03-28 17:45:17,580 INFO sqlalchemy.engine.Engine [cached since 0.01456s ago] ('ACCESSORY',)


['phone screen protector', 'headphone', 'memory card 256GB']

Combining multiple conditions using `Select.where()`:

In [239]:
stmt = (
    select(Product.product_name)
    .where(Product.type == ProductType.ACCESSORY)
    .where(Product.unit_price < 10)
)

In [240]:
print(stmt)

SELECT product.product_name 
FROM product 
WHERE product.type = :type_1 AND product.unit_price < :unit_price_1


In [241]:
print(stmt.compile().params)

{'type_1': <ProductType.ACCESSORY: 1>, 'unit_price_1': 10}


In [242]:
session.scalars(stmt).all()

2024-03-28 17:45:17,600 INFO sqlalchemy.engine.Engine SELECT product.product_name 
FROM product 
WHERE product.type = ? AND product.unit_price < ?
2024-03-28 17:45:17,600 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ('ACCESSORY', 10)


['phone screen protector']

Get all managers who oversee at least one employee (EXISTS):

In [243]:
stmt = (
    select(Employee)
    .where(Employee.employees.any())
)

Think of "employee" as the manager table and "employee_1" as the employee table:

In [244]:
print(stmt)

SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE EXISTS (SELECT 1 
FROM employee AS employee_1 
WHERE employee.employee_id = employee_1.manager_id)


The list of managers who have at least one subordinate:

In [245]:
for employee in session.scalars(stmt):
    print(employee)

2024-03-28 17:45:17,617 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee 
WHERE EXISTS (SELECT 1 
FROM employee AS employee_1 
WHERE employee.employee_id = employee_1.manager_id)
2024-03-28 17:45:17,617 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()
Employee(employee_id=1, manager_id=None, name='Alice', is_manager=True, hire_date=datetime.date(2024, 3, 27))
Employee(employee_id=4, manager_id=None, name='Louis', is_manager=True, hire_date=datetime.date(2024, 2, 27))


Conjunctions: `and_()`, `or_()`:

In [246]:
from sqlalchemy import and_, or_

Conjunctions using `AND`:

In [247]:
stmt = (
    select(Product.product_name)
    .where(
        and_(
            Product.type == ProductType.ACCESSORY,
            Product.unit_price < 10,
        )
    )
)

In [248]:
print(stmt)

SELECT product.product_name 
FROM product 
WHERE product.type = :type_1 AND product.unit_price < :unit_price_1


In [249]:
print(stmt.compile().params)

{'type_1': <ProductType.ACCESSORY: 1>, 'unit_price_1': 10}


In [250]:
session.scalars(stmt).all()

2024-03-28 17:45:17,648 INFO sqlalchemy.engine.Engine SELECT product.product_name 
FROM product 
WHERE product.type = ? AND product.unit_price < ?
2024-03-28 17:45:17,649 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('ACCESSORY', 10)


['phone screen protector']

Conjunctions using `OR`:

Employees who are either managers or employees supervised by a manager:

In [251]:
stmt = (
    select(Employee.employee_id, Employee.name)
    .where(
        or_(
            Employee.is_manager == True,
            Employee.manager != None,
        )
    )
)

In [252]:
print(stmt)

SELECT employee.employee_id, employee.name 
FROM employee 
WHERE employee.is_manager = true OR employee.manager_id IS NOT NULL


In [253]:
for row in session.execute(stmt):
    print(row)

2024-03-28 17:45:17,673 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.name 
FROM employee 
WHERE employee.is_manager = 1 OR employee.manager_id IS NOT NULL
2024-03-28 17:45:17,673 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ()
(1, 'Alice')
(2, 'Bob')
(3, 'Cathy')
(4, 'Louis')
(5, 'Lilly')


### Joins

Listing order details using `Select.join()`:

In [254]:
order_id = 1

In [255]:
stmt = (
    select(Product.product_name, Product.unit_price, OrderDetail.quantity)
    .join(OrderDetail.product)
    .join(OrderDetail.order)
    .where(Order.order_id == order_id)
)

In [256]:
print(stmt)

SELECT product.product_name, product.unit_price, order_detail.quantity 
FROM order_detail JOIN product ON product.product_id = order_detail.product_id JOIN "order" ON "order".order_id = order_detail.order_id 
WHERE "order".order_id = :order_id_1


In [257]:
for row in session.execute(stmt):
    print(row)

2024-03-28 17:45:17,693 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price, order_detail.quantity 
FROM order_detail JOIN product ON product.product_id = order_detail.product_id JOIN "order" ON "order".order_id = order_detail.order_id 
WHERE "order".order_id = ?
2024-03-28 17:45:17,694 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (1,)
('phone', Decimal('300.00'), 1)
('phone screen protector', Decimal('9.50'), 1)
('headphone', Decimal('25.99'), 1)


Listing order details using `Select.select_from()`, `Select.join()`:

In [258]:
stmt = (
    select(Product.product_name, Product.unit_price, OrderDetail.quantity)
    .select_from(Order)
    .join(Order.order_details)
    .join(OrderDetail.product)
    .where(Order.order_id == order_id)
)

In [259]:
print(stmt)

SELECT product.product_name, product.unit_price, order_detail.quantity 
FROM "order" JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = :order_id_1


In [260]:
for row in session.execute(stmt):
    print(row)

2024-03-28 17:45:17,708 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price, order_detail.quantity 
FROM "order" JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = ?
2024-03-28 17:45:17,709 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (1,)
('phone', Decimal('300.00'), 1)
('phone screen protector', Decimal('9.50'), 1)
('headphone', Decimal('25.99'), 1)


Listing order details using `Select.join_from()`:

In [261]:
stmt = (
    select(Product.product_name, Product.unit_price, OrderDetail.quantity)
    .join_from(Order, OrderDetail)
    .join_from(OrderDetail, Product)
    .where(Order.order_id == order_id)
)

In [262]:
print(stmt)

SELECT product.product_name, product.unit_price, order_detail.quantity 
FROM "order" JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = :order_id_1


In [263]:
for row in session.execute(stmt):
    print(row)

2024-03-28 17:45:17,723 INFO sqlalchemy.engine.Engine SELECT product.product_name, product.unit_price, order_detail.quantity 
FROM "order" JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".order_id = ?
2024-03-28 17:45:17,724 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (1,)
('phone', Decimal('300.00'), 1)
('phone screen protector', Decimal('9.50'), 1)
('headphone', Decimal('25.99'), 1)


### Self-Join and Alias

In [264]:
from sqlalchemy.orm import aliased

In [265]:
Manager = aliased(Employee, name="manager")

In [266]:
stmt = (
    select(
        Employee.name,
        Employee.employee_id,
        Manager.name.label("manager_name"),
        Manager.employee_id.label("manager_id"),
    )
    .join(Manager, Employee.manager_id == Manager.employee_id)
    .where(Employee.manager_id != None)
)

In [267]:
print(stmt)

SELECT employee.name, employee.employee_id, manager.name AS manager_name, manager.employee_id AS manager_id 
FROM employee JOIN employee AS manager ON employee.manager_id = manager.employee_id 
WHERE employee.manager_id IS NOT NULL


In [268]:
for row in session.execute(stmt):
    print(
        f"{row.name}#{row.employee_id} "
        f"reports to {row.manager_name}#{row.manager_id}."
    )

2024-03-28 17:45:17,748 INFO sqlalchemy.engine.Engine SELECT employee.name, employee.employee_id, manager.name AS manager_name, manager.employee_id AS manager_id 
FROM employee JOIN employee AS manager ON employee.manager_id = manager.employee_id 
WHERE employee.manager_id IS NOT NULL
2024-03-28 17:45:17,749 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()
Bob#2 reports to Alice#1.
Cathy#3 reports to Alice#1.
Lilly#5 reports to Louis#4.


### Grouping

In [269]:
from sqlalchemy.sql.functions import count
from sqlalchemy import desc

Displaying customers and the number of their orders:

In [270]:
min_count = 0

In [271]:
stmt = (
    select(
        Customer.first_name,
        Customer.last_name,
        count(Order.order_id).label("count"),
    )
    .join(Customer.orders)
    .group_by(Customer.customer_id)
    .having(count(Order.order_id) > min_count)  # by default, min_count = 0
    .order_by(desc("count"))
)

In [272]:
print(stmt)

SELECT customer.first_name, customer.last_name, count("order".order_id) AS count 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id GROUP BY customer.customer_id 
HAVING count("order".order_id) > :count_1 ORDER BY count DESC


In [273]:
for row in session.execute(stmt):
    orders = "order" if (row.count == 1) else "orders"
    print(f"{row.first_name} {row.last_name} has {row.count} {orders}.")

2024-03-28 17:45:17,772 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, count("order".order_id) AS count 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id GROUP BY customer.customer_id 
HAVING count("order".order_id) > ? ORDER BY count DESC
2024-03-28 17:45:17,772 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (0,)
Alex Smith has 2 orders.
Mary Taylor has 1 order.


### Conditional Queries

In [274]:
from sqlalchemy.orm import Session

In [275]:
def list_customer_orders(
        session: Session,
        name: str | None = None,
        is_shipped: bool = False,
        details: bool = True
):
    stmt = (
        select(
            Customer.first_name,
            Customer.last_name,
            Order.order_id,
            Order.is_shipped,
        )
        .join(Customer.orders)
        .where(Order.is_shipped == is_shipped)
    )

    if details:
        stmt = stmt.add_columns(
            Product.product_name,
            OrderDetail.quantity,
            Product.unit_price,
        ).join(Order.order_details).join(OrderDetail.product)

    if name is not None:
        stmt = stmt.where(Customer.first_name == name)

    for row in session.execute(stmt):
        print(row)

Orders for all customers that are not shipped, with details:

In [276]:
list_customer_orders(session, None, is_shipped=False, details=True)

2024-03-28 17:45:17,788 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped, product.product_name, order_detail.quantity, product.unit_price 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".is_shipped = 0
2024-03-28 17:45:17,789 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ()
('Alex', 'Smith', 1, False, 'phone', 1, Decimal('300.00'))
('Alex', 'Smith', 1, False, 'phone screen protector', 1, Decimal('9.50'))
('Alex', 'Smith', 1, False, 'headphone', 1, Decimal('25.99'))
('Alex', 'Smith', 2, False, 'memory card 256GB', 1, Decimal('21.99'))
('Mary', 'Taylor', 3, False, 'digital camera', 1, Decimal('45.99'))
('Mary', 'Taylor', 3, False, 'memory card 256GB', 2, Decimal('21.99'))


Orders for Alex that are not shipped, with details:

In [277]:
list_customer_orders(session, "Alex", is_shipped=False, details=True)

2024-03-28 17:45:17,795 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped, product.product_name, order_detail.quantity, product.unit_price 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id JOIN order_detail ON "order".order_id = order_detail.order_id JOIN product ON product.product_id = order_detail.product_id 
WHERE "order".is_shipped = 0 AND customer.first_name = ?
2024-03-28 17:45:17,796 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ('Alex',)
('Alex', 'Smith', 1, False, 'phone', 1, Decimal('300.00'))
('Alex', 'Smith', 1, False, 'phone screen protector', 1, Decimal('9.50'))
('Alex', 'Smith', 1, False, 'headphone', 1, Decimal('25.99'))
('Alex', 'Smith', 2, False, 'memory card 256GB', 1, Decimal('21.99'))


Orders for Alex that are not shipped, without details:

In [278]:
list_customer_orders(session, "Alex", is_shipped=False, details=False)

2024-03-28 17:45:17,801 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id 
WHERE "order".is_shipped = 0 AND customer.first_name = ?
2024-03-28 17:45:17,801 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Alex',)
('Alex', 'Smith', 1, False)
('Alex', 'Smith', 2, False)


Orders for Alex that are shipped, without details (none should be listed):

In [279]:
list_customer_orders(session, "Alex", is_shipped=True, details=False)

2024-03-28 17:45:17,806 INFO sqlalchemy.engine.Engine SELECT customer.first_name, customer.last_name, "order".order_id, "order".is_shipped 
FROM customer JOIN "order" ON customer.customer_id = "order".customer_id 
WHERE "order".is_shipped = 1 AND customer.first_name = ?
2024-03-28 17:45:17,807 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ('Alex',)


### Raw Queries

In [280]:
from sqlalchemy import text

In [281]:
clause = text("SELECT customer_id, first_name FROM customer")

In [282]:
clause

In [283]:
for row in session.execute(clause):
    print(row.customer_id, row.first_name)

2024-03-28 17:45:17,827 INFO sqlalchemy.engine.Engine SELECT customer_id, first_name FROM customer
2024-03-28 17:45:17,827 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()
1 Alex
2 Mary


## Updating Data

Updates using attributes of ORM-mapped objects:

In [284]:
mary = session.execute(
    select(Customer)
    .filter_by(first_name="Mary")
).scalar_one()

2024-03-28 17:45:17,833 INFO sqlalchemy.engine.Engine SELECT customer.customer_id, customer.first_name, customer.last_name, customer.email 
FROM customer 
WHERE customer.first_name = ?
2024-03-28 17:45:17,834 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Mary',)


In [285]:
mary

2024-03-28 17:45:17,839 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,840 INFO sqlalchemy.engine.Engine [cached since 0.4594s ago] (2,)


Customer(customer_id=2, first_name='Mary', last_name='Taylor', address='200-139 Jefferson Street, NY', email='mary_taylor@test.com')

In [286]:
mary.last_name = "Smith"

In [287]:
print("Is instance modified:", session.is_modified(mary))  # True

Is instance modified: True


In [288]:
print("Is object dirty:", mary in session.dirty)  # True

Is object dirty: True


In [289]:
session.flush()

2024-03-28 17:45:17,859 INFO sqlalchemy.engine.Engine UPDATE customer SET last_name=? WHERE customer.customer_id = ?
2024-03-28 17:45:17,860 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('Smith', 2)


In [290]:
print("Is instance modified after flushing:", session.is_modified(mary))  # False

Is instance modified after flushing: False


In [291]:
print("Is object dirty after flushing:", mary in session.dirty)  # False

Is object dirty after flushing: False


In [292]:
mary

Customer(customer_id=2, first_name='Mary', last_name='Smith', address='200-139 Jefferson Street, NY', email='mary_taylor@test.com')

In [293]:
session.commit()

2024-03-28 17:45:17,882 INFO sqlalchemy.engine.Engine COMMIT


Updates using `WHERE`:

In [294]:
from sqlalchemy import update

In [295]:
stmt = (
    update(Customer)
    .where(Customer.first_name.in_(["Alex", "Mary"]))
    .filter_by(last_name="Smith")
    .values(last_name="Taylor")
    .returning(Customer)
)

In [296]:
print(stmt)

UPDATE customer SET last_name=:last_name WHERE customer.first_name IN (__[POSTCOMPILE_first_name_1]) AND customer.last_name = :last_name_1 RETURNING customer.customer_id, customer.first_name, customer.last_name, customer.email


In [297]:
print(stmt.compile().params)

{'last_name': 'Taylor', 'first_name_1': ['Alex', 'Mary'], 'last_name_1': 'Smith'}


In [298]:
customers = session.scalars(stmt)

2024-03-28 17:45:17,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,914 INFO sqlalchemy.engine.Engine UPDATE customer SET last_name=? WHERE customer.first_name IN (?, ?) AND customer.last_name = ? RETURNING customer_id, first_name, last_name, email
2024-03-28 17:45:17,914 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ('Taylor', 'Alex', 'Mary', 'Smith')


In [299]:
for customer in customers:
    print(customer)

2024-03-28 17:45:17,920 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,921 INFO sqlalchemy.engine.Engine [cached since 0.5403s ago] (1,)
Customer(customer_id=1, first_name='Alex', last_name='Taylor', address='618 Oak Lane, CA', email='alex_smith@test.com')
2024-03-28 17:45:17,922 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,922 INFO sqlalchemy.engine.Engine [cached since 0.5417s ago] (2,)
Customer(customer_id=2, first_name='Mary', last_name='Taylor', address='200-139 Jefferson Street, NY', email='mary_taylor@test.com')


In [300]:
session.commit()

2024-03-28 17:45:17,929 INFO sqlalchemy.engine.Engine COMMIT


ORM bulk update by primary key:

In [301]:
session.execute(
    update(Customer),
    [
        {"customer_id": 1, "last_name": "Smith"},
        {"customer_id": 2, "last_name": "Taylor"},
    ],
)

2024-03-28 17:45:17,937 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,939 INFO sqlalchemy.engine.Engine UPDATE customer SET last_name=? WHERE customer.customer_id = ?
2024-03-28 17:45:17,940 INFO sqlalchemy.engine.Engine [generated in 0.00084s] [('Smith', 1), ('Taylor', 2)]


In [302]:
session.commit()

2024-03-28 17:45:17,946 INFO sqlalchemy.engine.Engine COMMIT


In [303]:
customers = session.scalars(select(Customer))
for customer in customers:
    print(customer)

2024-03-28 17:45:17,955 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,956 INFO sqlalchemy.engine.Engine SELECT customer.customer_id, customer.first_name, customer.last_name, customer.email 
FROM customer
2024-03-28 17:45:17,956 INFO sqlalchemy.engine.Engine [cached since 0.5816s ago] ()
2024-03-28 17:45:17,958 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,958 INFO sqlalchemy.engine.Engine [cached since 0.5777s ago] (1,)
Customer(customer_id=1, first_name='Alex', last_name='Smith', address='618 Oak Lane, CA', email='alex_smith@test.com')
2024-03-28 17:45:17,960 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-28 17:45:17,960 INFO sqlalchemy.engine.Engine [cached since 0.5794s ago] (2,)
Customer(customer_id=2, first_name='Mary', last_name='Taylor', address='200-139 Jefferson Street, NY', email='

### Differences in Update Approaches: Class Attributes vs. Instance Attributes

Updating with class attributes (increment the inventory by one):

In [304]:
product = session.get(Product, 1)

2024-03-28 17:45:17,967 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-28 17:45:17,968 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (1,)


In [305]:
product.units_in_stock = Product.units_in_stock + 1

In [306]:
session.commit()

2024-03-28 17:45:17,980 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock + ?) WHERE product.product_id = ?
2024-03-28 17:45:17,980 INFO sqlalchemy.engine.Engine [generated in 0.00089s] (1, 1)
2024-03-28 17:45:17,982 INFO sqlalchemy.engine.Engine COMMIT


Updating with instance attributes (decrement the inventory by one):

In [307]:
product = session.get(Product, 1)

2024-03-28 17:45:17,989 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:17,990 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-28 17:45:17,990 INFO sqlalchemy.engine.Engine [cached since 0.02352s ago] (1,)


In [308]:
product.units_in_stock -= 1

In [309]:
session.commit()

2024-03-28 17:45:18,002 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=? WHERE product.product_id = ?
2024-03-28 17:45:18,002 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (5, 1)
2024-03-28 17:45:18,003 INFO sqlalchemy.engine.Engine COMMIT


## Deleting Data

In [310]:
from sqlalchemy import delete

To enable foreign key constraints in SQLite, you need to
[activate foreign key support](https://docs.sqlalchemy.org/en/20/dialects/sqlite.html#foreign-key-support):


In [311]:
from sqlalchemy.engine import Engine
from sqlalchemy import event

@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

Delete the first order from our system using an ORM-mapped instance:

In [312]:
order = session.get(Order, 1)

2024-03-28 17:45:18,021 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:18,023 INFO sqlalchemy.engine.Engine SELECT "order".order_id AS order_order_id, "order".customer_id AS order_customer_id, "order".employee_id AS order_employee_id, "order".order_datetime AS order_order_datetime, "order".is_shipped AS order_is_shipped 
FROM "order" 
WHERE "order".order_id = ?
2024-03-28 17:45:18,023 INFO sqlalchemy.engine.Engine [generated in 0.00072s] (1,)


In [313]:
if order is not None:
    session.delete(order)
    print("Order in session after delete?", order in session)  # True
    session.commit()
    print("Order in session after committing?", order in session)  # False

Order in session after delete? True
2024-03-28 17:45:18,030 INFO sqlalchemy.engine.Engine DELETE FROM "order" WHERE "order".order_id = ?
2024-03-28 17:45:18,031 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (1,)
2024-03-28 17:45:18,032 INFO sqlalchemy.engine.Engine COMMIT
Order in session after committing? False


Checking results (orders):

In [314]:
for order in session.scalars(select(Order)):
    print(order)

2024-03-28 17:45:18,040 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:18,041 INFO sqlalchemy.engine.Engine SELECT "order".order_id, "order".customer_id, "order".employee_id, "order".order_datetime, "order".is_shipped 
FROM "order"
2024-03-28 17:45:18,041 INFO sqlalchemy.engine.Engine [cached since 0.6173s ago] ()
Order(order_id=2, customer_id=1, employee_id=None, order_datetime=datetime.datetime(2024, 3, 28, 17, 45, 17, 401588), is_shipped=False)
Order(order_id=3, customer_id=2, employee_id=None, order_datetime=datetime.datetime(2024, 3, 28, 17, 45, 17, 409931), is_shipped=False)


Check order details:

In [315]:
for od in session.scalars(select(OrderDetail)):
    print(od)

2024-03-28 17:45:18,048 INFO sqlalchemy.engine.Engine SELECT order_detail.order_id, order_detail.product_id, order_detail.quantity 
FROM order_detail
2024-03-28 17:45:18,048 INFO sqlalchemy.engine.Engine [cached since 0.6145s ago] ()
OrderDetail(order_id=2, product_id=5, quantity=1)
OrderDetail(order_id=3, product_id=4, quantity=1)
OrderDetail(order_id=3, product_id=5, quantity=2)


Delete all the remaining orders using the `WHERE` clause:


In [316]:
stmt = delete(Order).where(Order.order_id != 1)

In [317]:
print(stmt)

DELETE FROM "order" WHERE "order".order_id != :order_id_1


In [318]:
session.execute(stmt)

2024-03-28 17:45:18,066 INFO sqlalchemy.engine.Engine DELETE FROM "order" WHERE "order".order_id != ?
2024-03-28 17:45:18,067 INFO sqlalchemy.engine.Engine [generated in 0.00089s] (1,)


In [319]:
session.commit()

2024-03-28 17:45:18,073 INFO sqlalchemy.engine.Engine COMMIT


Checking orders:

In [320]:
for order in session.scalars(select(Order)):
    print(order)

2024-03-28 17:45:18,081 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-28 17:45:18,082 INFO sqlalchemy.engine.Engine SELECT "order".order_id, "order".customer_id, "order".employee_id, "order".order_datetime, "order".is_shipped 
FROM "order"
2024-03-28 17:45:18,082 INFO sqlalchemy.engine.Engine [cached since 0.6582s ago] ()


And order details:

In [321]:
for od in session.scalars(select(OrderDetail)):
    print(od)

2024-03-28 17:45:18,088 INFO sqlalchemy.engine.Engine SELECT order_detail.order_id, order_detail.product_id, order_detail.quantity 
FROM order_detail
2024-03-28 17:45:18,088 INFO sqlalchemy.engine.Engine [cached since 0.6547s ago] ()


Finally, close the session:

In [322]:
session.close()

2024-03-28 17:45:18,093 INFO sqlalchemy.engine.Engine ROLLBACK
